In [1]:
import os
import cv2
import sys
import json
import numpy
from darkflow.net.build import TFNet
print('python version:', sys.version)
print('cv2 version:', cv2.__version__)
## tensorflow-gpu 
## CUDA 7.5

python version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
cv2 version: 3.3.0


In [2]:
# ckeck tensorflow gpu
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# ---
# What returns in cmd
# reference: https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell
# 2017-11-02 23:23:21.039622: I tensorflow/core/common_runtime/direct_session.cc:300] Device mapping:
# /job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GeForce 920M, pci bus id: 0000:04:00.0


### Problems

In [3]:
# Tensorflow with gpu then [ "gpu": 0.7 will be working. ]
# Reference: https://www.google.com.tw/search?q=darkflow+gpu&oq=darkflow+gpu&aqs=chrome..69i57.3855j0j7&sourceid=chrome&ie=UTF-8

# >>> conda unistall tensorflow: 1.3.0-py35_0 conda-forge
# >>> conda install tensorflow-gpu or pip install tensorflow-gpu

In [4]:
# CUDA with opencv? 
# https://jamesbowley.co.uk/buildcompile-opencv-v3-3-on-windows-with-cuda-8-0-and-intel-mkltbb/ 看不懂！？
# https://www.scivision.co/anaconda-python-opencv3/ 

# print(cv2.getBuildInformation())

#####################################################################################################################

### TFNet Setting

In [5]:
# Set TFNet initiail options
options = {"model": "cfg/yolo.cfg",
           "load": "bin/yolo.weights",
           "threshold": 0.2,
           "gpu": 0.9}

tfnet = TFNet(options)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 269862452 bytes
Finished in 0.028172969818115234s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Load  |  Yep

### Video Object Detection with Darkflow

In [6]:
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html
# cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('/home/pikajiyu/Downloads/vision_api-master/opencv_practice/test.avi')
# cap = cv2.VideoCapture('./Funny.mp4')
cap = cv2.VideoCapture('./test.mpg')

tag_list = []
i = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check time 
    e1 = cv2.getTickCount()

    if (i%50) == 0:

        # Prediction with tfnet 
        print('=== Start Predicting with darkflow ===')
        predictions = tfnet.return_predict(frame)
        for x in range(1,len(predictions)):
                        if predictions!=[]:
                            topleftx = predictions[x]['topleft']['x']
                            toplefty = predictions[x]['topleft']['y']
                            bottomrightx = predictions[x]['bottomright']['x']
                            bottomrighty = predictions[x]['bottomright']['y']
                            detect = predictions[x]['label']
                            if detect not in tag_list:
                                tag_list.append(detect)
                            confidence = str(predictions[x]['confidence'])
                            font = cv2.FONT_HERSHEY_SIMPLEX
                            
    cv2.rectangle(frame,(topleftx,toplefty),(bottomrightx,bottomrighty),(0,255,0),3)
    cv2.putText(frame,detect,(bottomrightx-150,bottomrighty-60), font, 2,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(frame,confidence,(bottomrightx-150,bottomrighty-20), font, 1,(80,50,255),2,cv2.LINE_AA)
                            
    # Display the resulting frame
    cv2.imshow('frame',frame)
    
    # Check time spent
    e2 = cv2.getTickCount()
    print('Cost time:',(e2 - e1)/cv2.getTickFrequency())
    
    # Skip frame 
    i += 1
    
    # Quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print Tag List
print("=== Object Detec ===")
print(tag_list)
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

=== Start Predicting with darkflow ===
Cost time: 4.524212836
Cost time: 0.00169975
Cost time: 0.00100872
Cost time: 0.001159953
Cost time: 0.001061112
Cost time: 0.000935262
Cost time: 0.000937347
Cost time: 0.001052624
Cost time: 0.000939423
Cost time: 0.001146437
Cost time: 0.0012674
Cost time: 0.001130231
Cost time: 0.000970998
Cost time: 0.001278365
Cost time: 0.001452892
Cost time: 0.001328845
Cost time: 0.001210612
Cost time: 0.001160994
Cost time: 0.001091204
Cost time: 0.001833652
Cost time: 0.001019886
Cost time: 0.001375873
Cost time: 0.000965101
Cost time: 0.001559984
Cost time: 0.000905721
Cost time: 0.001048228
Cost time: 0.0010435
Cost time: 0.00141622
Cost time: 0.000925327
Cost time: 0.000906953
Cost time: 0.001001954
Cost time: 0.001640067
Cost time: 0.001089741
Cost time: 0.001194846
Cost time: 0.001372313
Cost time: 0.001144868
Cost time: 0.001271965
Cost time: 0.001247109
Cost time: 0.001154339
Cost time: 0.001380881
Cost time: 0.001367475
Cost time: 0.001524947
Co

error: /feedstock_root/build_artefacts/opencv_1506439680421/work/opencv-3.3.0/modules/highgui/src/window.cpp:325: error: (-215) size.width>0 && size.height>0 in function imshow


In [ ]:
# conda config --add channels conda-forge
# conda install --use-local opencv
# sudo apt-get install libavcodec-dev libavformat-dev libavdevice-dev
# sudo apt-get install pkg-config
# sudo apt-get install libgtk2.0-dev



# problem issue # opencv Unable to stop the stream: Inappropriate ioctl for device
# https://stackoverflow.com/questions/42562876/opencv3-error-unable-to-stop-the-stream-inappropriate-ioctl-for-device
# https://stackoverflow.com/questions/41200201/opencv-unable-to-stop-the-stream-inappropriate-ioctl-for-device


# OR JUST INSTALL cv2 version: 3.3.0
# 3.3 problem # scn == 3 || scn == 4 in function cvtColor
# https://github.com/HackerHouseYT/AI-Smart-Mirror/issues/36